In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
pd.options.mode.chained_assignment = None

In [2]:
doge = pd.read_csv('API Data/DOGE_price.csv',index_col = 0)
doge = doge[~doge.index.duplicated()].reset_index()
doge

,time,open,high,low,close,volume,num_trades
0,2019-07-05 07:00:00,0.004490,0.004600,0.003760,0.003925,949430724.0,6487
1,2019-07-05 08:00:00,0.003915,0.003928,0.003765,0.003835,159004267.0,1529
2,2019-07-05 09:00:00,0.003845,0.003920,0.003814,0.003841,105517060.0,1241
3,2019-07-05 10:00:00,0.003837,0.003869,0.003710,0.003738,100804728.0,1213
4,2019-07-05 11:00:00,0.003728,0.003734,0.003636,0.003666,120261079.0,1416
...,...,...,...,...,...,...,...
17399,2021-06-30 21:00:00,0.251820,0.252080,0.244850,0.245520,66497079.3,22834
17400,2021-06-30 22:00:00,0.245550,0.247050,0.242860,0.243060,48553525.7,17177
17401,2021-06-30 23:00:00,0.243050,0.247150,0.242300,0.246970,44887310.7,16267
17402,2021-07-01 00:00:00,0.246970,0.248000,0.244770,0.247020,35714148.5,15537


In [3]:
all_times = pd.date_range(start = doge.time.min(), end = doge.time.max(), freq = '1H')
all_times = pd.DataFrame(all_times)
all_times['time'] = ''
for i in range(len(all_times[0])):
    all_times['time'][i] = str(all_times[0][i])
all_times.drop(0, axis = 1,inplace = True)

In [4]:
doge = pd.merge(all_times,doge, on = "time", how = 'left')
doge.fillna(method="bfill",inplace = True)

In [6]:
all_times

,time
0,2019-07-05 07:00:00
1,2019-07-05 08:00:00
2,2019-07-05 09:00:00
3,2019-07-05 10:00:00
4,2019-07-05 11:00:00
...,...
17438,2021-06-30 21:00:00
17439,2021-06-30 22:00:00
17440,2021-06-30 23:00:00
17441,2021-07-01 00:00:00


In [7]:
doge['200_EMA'] = doge.close.ewm(span=4800,adjust = False).mean()
doge['50_EMA'] = doge.close.ewm(span=1200, adjust = False).mean()
doge['12_EMA'] = doge.close.ewm(span=288,adjust = False).mean()
doge['26_EMA'] = doge.close.ewm(span=624, adjust = False).mean()
doge['20_SMA'] = doge.close.rolling(window=480).mean()
doge['MACD'] = doge['12_EMA'] - doge['26_EMA']

In [8]:
doge['price_dif_200EMA'] = doge.close - doge['200_EMA']
doge['price_dif_50EMA'] = doge.close - doge['50_EMA']
doge['OBV'] = 0
for i in range(24,len(doge)):
    if doge.close[i] > doge.close[i-24]:
        doge.OBV[i] = doge.volume[i]
    else: 
        doge.OBV[i] = -doge.volume[i]

In [10]:
delta = doge.close.diff()
up = delta.clip(lower=0)
down = -1*delta.clip(upper=0)
ema_up = up.ewm(com=13, adjust=False).mean()
ema_down = down.ewm(com=13, adjust=False).mean()
doge['RSI'] = ema_up/ema_down

In [11]:
def bb(data, sma, window):
    std = data.rolling(window = window).std()
    upper_bb = sma + std * 2
    lower_bb = sma - std * 2
    return upper_bb, lower_bb

doge['upper_bb'], doge['lower_bb'] = bb(doge.close, doge['20_SMA'], 480)
doge['price_dif_upper_bb'] = doge.upper_bb - doge.close
doge['price_dif_lower_bb'] = doge.lower_bb - doge.close
doge

,time,open,high,low,close,volume,num_trades,200_EMA,50_EMA,12_EMA,...,20_SMA,MACD,price_dif_200EMA,price_dif_50EMA,OBV,upper_bb,lower_bb,price_dif_upper_bb,price_dif_lower_bb,RSI
0,2019-07-05 07:00:00,0.004490,0.004600,0.003760,0.003925,949430724.0,6487.0,0.003925,0.003925,0.003925,...,NaN,0.000000e+00,0.000000,0.000000,0,NaN,NaN,NaN,NaN,NaN
1,2019-07-05 08:00:00,0.003915,0.003928,0.003765,0.003835,159004267.0,1529.0,0.003925,0.003925,0.003924,...,NaN,-3.333492e-07,-0.000090,-0.000089,0,NaN,NaN,NaN,NaN,0.000000
2,2019-07-05 09:00:00,0.003845,0.003920,0.003814,0.003841,105517060.0,1241.0,0.003925,0.003925,0.003924,...,NaN,-6.410023e-07,-0.000084,-0.000083,0,NaN,NaN,NaN,NaN,0.005151
3,2019-07-05 10:00:00,0.003837,0.003869,0.003710,0.003738,100804728.0,1213.0,0.003925,0.003924,0.003922,...,NaN,-1.330225e-06,-0.000187,-0.000186,0,NaN,NaN,NaN,NaN,0.004702
4,2019-07-05 11:00:00,0.003728,0.003734,0.003636,0.003666,120261079.0,1416.0,0.003925,0.003924,0.003921,...,NaN,-2.278469e-06,-0.000258,-0.000257,0,NaN,NaN,NaN,NaN,0.004415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17438,2021-06-30 21:00:00,0.251820,0.252080,0.244850,0.245520,66497079.3,22834.0,0.201257,0.303151,0.258650,...,0.273843,-2.580037e-02,0.044263,-0.057631,-66497079,0.352329,0.195356,0.106809,-0.050164,0.693459
17439,2021-06-30 22:00:00,0.245550,0.247050,0.242860,0.243060,48553525.7,17177.0,0.201275,0.303051,0.258542,...,0.273673,-2.577581e-02,0.041785,-0.059991,-48553525,0.352070,0.195275,0.109010,-0.047785,0.620848
17440,2021-06-30 23:00:00,0.243050,0.247150,0.242300,0.246970,44887310.7,16267.0,0.201294,0.302958,0.258462,...,0.273507,-2.573638e-02,0.045676,-0.055988,-44887310,0.351794,0.195220,0.104824,-0.051750,0.800078
17441,2021-07-01 00:00:00,0.246970,0.248000,0.244770,0.247020,35714148.5,15537.0,0.201313,0.302865,0.258383,...,0.273347,-2.569660e-02,0.045707,-0.055845,-35714148,0.351537,0.195158,0.104517,-0.051862,0.802546
